In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
postalcodes = pd.DataFrame(columns=column_names)

In [5]:
postalcodes

,PostalCode,Borough,Neighborhood


In [6]:
from bs4 import BeautifulSoup

In [7]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [8]:
# Set headers
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [9]:
req = requests.get(url, headers)

In [10]:
soup = BeautifulSoup(req.content, 'html.parser')

In [13]:
# print(soup.prettify())

In [11]:
codes = soup.select("td p b")

In [16]:
bn = soup.select("td p span")

In [17]:
for index, code in enumerate(codes):
    if not bn[index].find('i'):
        code = code.get_text()
        borough_and_neighborhood = bn[index].get_text(separator=" ").strip().split('(')
        borough = borough_and_neighborhood[0].split('/')
        
        if len(borough) > 1:
            neighborhood = borough[1]
        else:
            neighborhood = borough_and_neighborhood[1].replace(')', '').replace('  ', ' ').replace(' /', ',')
        
        borough = borough[0]
        
        # print(code, borough, neighborhood)
        
        postalcodes = postalcodes.append({'PostalCode': code, 'Borough': borough, 'Neighborhood': neighborhood}, ignore_index=True)

In [43]:
postalcodes.head()

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [44]:
postalcodes.shape

(103, 3)

In [20]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [27]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
postalcodes = postalcodes.sort_values(by=['PostalCode'])

In [45]:
postalcodes.head()

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [55]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
postalcodes = postalcodes.merge(coordinates, left_on='PostalCode', right_on='Postal Code')

In [62]:
postalcodes = postalcodes.drop(columns=['Postal Code'])

In [63]:
postalcodes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
